<a href="https://colab.research.google.com/github/ShannonReyon/RTOS-LAB4/blob/main/data_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download brsdincer/vehicle-detection-image-set

 87% 104M/119M [00:00<00:00, 190MB/s] 
100% 119M/119M [00:00<00:00, 182MB/s]


In [ ]:
! unzip vehicle-detection-image-set.zip

In [ ]:
import os
import time
import math
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
%matplotlib inline

In [ ]:
VEHICLES_DIR = '/content/data/vehicles'
NON_VEHICLES_DIR = '/content/data/non-vehicles'

In [ ]:
vehicles_image_names = os.listdir(VEHICLES_DIR)
non_vehicles_image_names = os.listdir(NON_VEHICLES_DIR)

In [ ]:
print('TOTAL NO. OF NON-VEHICLE IMAGES:', len(non_vehicles_image_names))
print('TOTAL NO. OF VEHICLE IMAGES:', len(vehicles_image_names))

TOTAL NO. OF NON-VEHICLE IMAGES: 8968
TOTAL NO. OF VEHICLE IMAGES: 8792


In [ ]:
print(vehicles_image_names[:10])
print(non_vehicles_image_names[:10])

['1279.png', '924.png', 'right (162).png', '1881.png', 'left (21).png', '3881.png', '4204.png', '3481.png', 'left (47).png', '5858.png']
['extra1278.png', 'extra4423.png', 'extra1916.png', 'image503.png', 'image1337.png', 'image2862.png', 'extra3752.png', 'extra3545.png', 'image401.png', 'extra5207.png']


In [ ]:
BASE_DIR = '../tmp/'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TEST_DIR = os.path.join(BASE_DIR, 'test')
VEHICLES_TRAIN_DIR = os.path.join(TRAIN_DIR, 'vehicles')
NON_VEHICLES_TRAIN_DIR = os.path.join(TRAIN_DIR, 'non-vehicles')
VEHICLES_TEST_DIR = os.path.join(TEST_DIR, 'vehicles')
NON_VEHICLES_TEST_DIR = os.path.join(TEST_DIR, 'non-vehicles')

In [ ]:
TRAIN_SIZE = 0.8
train_vehicles, test_vehicles = train_test_split(
    vehicles_image_names, train_size=TRAIN_SIZE, shuffle=True, random_state=1
)

train_non_vehicles, test_non_vehicles = train_test_split(
    non_vehicles_image_names, train_size=TRAIN_SIZE, shuffle=True, random_state=1
)


In [ ]:
print('TOTAL NO. OF TRAIN VEHICLE IMAGES:', len(train_vehicles))
print('TOTAL NO. OF TRAIN NON-VEHICLE IMAGES:', len(train_non_vehicles))

print('TOTAL NO. OF TEST VEHICLE IMAGES:', len(test_vehicles))
print('TOTAL NO. OF TEST NON-VEHICLE IMAGES:', len(test_non_vehicles))

TOTAL NO. OF TRAIN VEHICLE IMAGES: 7033
TOTAL NO. OF TRAIN NON-VEHICLE IMAGES: 7174
TOTAL NO. OF TEST VEHICLE IMAGES: 1759
TOTAL NO. OF TEST NON-VEHICLE IMAGES: 1794


In [ ]:
def move_images(image_list, old_dir_path, new_dir_path):
    if not os.path.exists(new_dir_path):
        os.makedirs(new_dir_path)
    
    for file_name in image_list:
        shutil.copy(
            os.path.join(old_dir_path, file_name),
            os.path.join(new_dir_path, file_name)
        )
    print(f'{len(image_list)} IMAGES COPIED TO {new_dir_path}')


In [ ]:
move_images(train_vehicles, VEHICLES_DIR, VEHICLES_TRAIN_DIR)
move_images(train_non_vehicles, NON_VEHICLES_DIR, NON_VEHICLES_TRAIN_DIR)

move_images(test_vehicles, VEHICLES_DIR, VEHICLES_TEST_DIR)
move_images(test_non_vehicles, NON_VEHICLES_DIR, NON_VEHICLES_TEST_DIR)

7033 IMAGES COPIED TO ../tmp/train/vehicles
7174 IMAGES COPIED TO ../tmp/train/non-vehicles
1759 IMAGES COPIED TO ../tmp/test/vehicles
1794 IMAGES COPIED TO ../tmp/test/non-vehicles


In [ ]:
len(os.listdir(VEHICLES_TRAIN_DIR))

7033

In [ ]:
IMG_SIZE = 256

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    class_mode='binary'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='binary'
)

Found 14207 images belonging to 2 classes.
Found 3553 images belonging to 2 classes.


In [ ]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPool2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPool2D(2, 2),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 246016)            0         
                                                                 
 dense (Dense)               (None, 1)                 2

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5
)

model_path = 'vehicle_detection.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    save_best_only=True
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=40,
    validation_data=test_generator,
    validation_steps=len(test_generator),
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/40
444/444 [==============================] - 787s 2s/step - loss: 0.1649 - accuracy: 0.9379 - val_loss: 0.0865 - val_accuracy: 0.9750
Epoch 2/40
444/444 [==============================] - 783s 2s/step - loss: 0.0636 - accuracy: 0.9784 - val_loss: 0.0623 - val_accuracy: 0.9795
Epoch 3/40
444/444 [==============================] - 783s 2s/step - loss: 0.0416 - accuracy: 0.9863 - val_loss: 0.0540 - val_accuracy: 0.9809
Epoch 4/40
444/444 [==============================] - 783s 2s/step - loss: 0.0260 - accuracy: 0.9921 - val_loss: 0.0492 - val_accuracy: 0.9837
Epoch 5/40
444/444 [==============================] - 787s 2s/step - loss: 0.0413 - accuracy: 0.9859 - val_loss: 0.0485 - val_accuracy: 0.9854
Epoch 6/40
444/444 [==============================] - 797s 2s/step - loss: 0.0232 - accuracy: 0.9920 - val_loss: 0.0955 - val_accuracy: 0.9690
Epoch 7/40
444/444 [==============================] - 779s 2s/step - loss: 0.0123 - accuracy: 0.9961 - val_loss: 0.0467 - val_accuracy: 0.9879

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

NameError: ignored